In [1]:
import numpy as np
import sympy as sp
from sympy import Matrix
from joblib import Parallel, delayed
import multiprocessing

In [2]:
# Function to generate a random invertible key matrix
def generate_random_key_matrix(n):
    while True:
        key = np.random.randint(0, 26, (n, n))
        if np.linalg.matrix_rank(key) == n and np.gcd(int(np.linalg.det(key)), 26) == 1:
            return key

In [3]:
# Function to encrypt a message using Hill Cipher
def hill_cipher_encrypt(plain_text, key):
    # Convert plain text to numbers
    plain_text = [ord(char) - ord('a') for char in plain_text.lower()]
    n = len(key)
    encrypted_text = ""

    # Pad plain text if its length is not a multiple of key size
    if len(plain_text) % n != 0:
        plain_text.extend([0] * (n - len(plain_text) % n))

    # Encrypt the text
    for i in range(0, len(plain_text), n):
        block = plain_text[i:i+n]
        encrypted_block = np.dot(key, block) % 26
        encrypted_text += ''.join([chr(char + ord('a')) for char in encrypted_block])

    return encrypted_text

In [4]:
# Function to decrypt a message using Hill Cipher
def hill_cipher_decrypt(encrypted_text, key):
    # Convert encrypted text to numbers
    encrypted_text = [ord(char) - ord('a') for char in encrypted_text.lower()]
    key_inv = Matrix(key).inv_mod(26)
    decrypted_text = ""

    # Decrypt the text
    for i in range(0, len(encrypted_text), len(key)):
        block = encrypted_text[i:i+len(key)]
        decrypted_block = (key_inv * Matrix(block)) % 26
        decrypted_text += ''.join([chr(int(char) + ord('a')) for char in decrypted_block])

    return decrypted_text

In [5]:
# Parallel version of Hill Cipher encryption
def parallel_hill_cipher_encrypt(plain_text, key, num_cores):
    split_text = [plain_text[i::num_cores] for i in range(num_cores)]
    encrypted_text_parts = Parallel(n_jobs=num_cores)(delayed(hill_cipher_encrypt)(text_part, key) for text_part in split_text)
    return ''.join(encrypted_text_parts)

In [6]:
# Parallel version of Hill Cipher decryption
def parallel_hill_cipher_decrypt(encrypted_text, key, num_cores):
    split_text = [encrypted_text[i::num_cores] for i in range(num_cores)]
    decrypted_text_parts = Parallel(n_jobs=num_cores)(delayed(hill_cipher_decrypt)(text_part, key) for text_part in split_text)
    return ''.join(decrypted_text_parts)

In [7]:
# Example usage
key = generate_random_key_matrix(2)
print("Generated Key Matrix:\n", key)
plain_text = "hello"
encrypted_text = parallel_hill_cipher_encrypt(plain_text, key, num_cores=4)
print("Encrypted Text:", encrypted_text)
decrypted_text = parallel_hill_cipher_decrypt(encrypted_text, key, num_cores=4)
print("Decrypted Text:", decrypted_text)

Generated Key Matrix:
 [[7 3]
 [6 7]]
Encrypted Text: nkcyzozo
Decrypted Text: yjqstruw


In [8]:
import time
import matplotlib.pyplot as plt

In [9]:
# Function to measure the time taken for serial encryption
def measure_serial_encryption_time(plain_text, key):
    start_time = time.time()
    encrypted_text = hill_cipher_encrypt(plain_text, key)
    end_time = time.time()
    return end_time - start_time

In [10]:
# Function to measure the time taken for parallel encryption
def measure_parallel_encryption_time(plain_text, key, num_cores):
    start_time = time.time()
    encrypted_text = parallel_hill_cipher_encrypt(plain_text, key, num_cores)
    end_time = time.time()
    return end_time - start_time

In [11]:
# Function to generate data for plotting
def generate_data():
    N_values = [100, 200, 300, 400, 500]  # Different values of N
    serial_times = []
    parallel_times = []

    for N in N_values:
        key = generate_random_key_matrix(N)
        plain_text = "hello" * (N // 5)  # Adjusting plain text length for each value of N
        serial_time = measure_serial_encryption_time(plain_text, key)
        parallel_time = measure_parallel_encryption_time(plain_text, key, num_cores=4)
        serial_times.append(serial_time)
        parallel_times.append(parallel_time)

    return N_values, serial_times, parallel_times

In [12]:
# Function to plot the graph for N vs Speed Up
def plot_speed_up_graph(N_values, serial_times, parallel_times):
    speed_up_data = [serial_times[i] / parallel_times[i] for i in range(len(N_values))]
    plt.figure(figsize=(8, 6))
    plt.plot(N_values, speed_up_data, marker='o', linestyle='-')
    plt.title('N vs Speed Up')
    plt.xlabel('N')
    plt.ylabel('Speed Up')
    plt.grid(True)
    plt.show()

In [13]:
# Function to plot the graph for N vs Parallel Efficiency
def plot_parallel_efficiency_graph(N_values, parallel_times):
    num_cores = 4  # Number of processing elements
    parallel_efficiency_data = [serial_times[i] / (parallel_times[i] * num_cores) for i in range(len(N_values))]
    plt.figure(figsize=(8, 6))
    plt.plot(N_values, parallel_efficiency_data, marker='o', linestyle='-')
    plt.title('N vs Parallel Efficiency')
    plt.xlabel('N')
    plt.ylabel('Parallel Efficiency')
    plt.grid(True)
    plt.show()

In [ ]:
# Generate data
N_values, serial_times, parallel_times = generate_data()

# Plot graphs
plot_speed_up_graph(N_values, serial_times, parallel_times)
plot_parallel_efficiency_graph(N_values, parallel_times)